# Manage connections and discover assets

This notebook contains steps and code to demonstrate support for connections in Cloud Pak for Data using
[`cpdctl`](https://github.com/IBM/cpdctl) CLI tool. 

Some familiarity with Python is helpful. This notebook uses Python 3.7.


## Table of contents

#### 1. Installation and Configuration of CPDCTL

    1.1 Install the latest version of cpdctl 
    1.2 Check cpdctl version 
    1.3 Add CPD cluster configuration 
    
    
#### 2. Demo cpdctl connection commands

    2.0 Create a project for connection or Choose an existing project
    2.1 Create a connection
    2.2 List connections
    2.3 Discover connection ad-hoc
    2.4 Discover data asset (not available until CPD 3.5)
    2.5 Get connection
    2.6 Update connection
    2.7 List actions
    2.8 Perform action
    2.9 Discover connection
    2.10 Delete connection
    2.11 Clean up - Delete project created at the beggining (containing our connection)
    
#### 3. Demo cpdctl connection datasource types commands

    3.1 List datasource types
    3.2 Get datasource type
    
### Another useful documentation regarding connections parameters can be found here:

Connections REST API docs https://api.dataplatform.dev.cloud.ibm.com/v2/connections/api/explorer/

Watson Data API docs https://cloud.ibm.com/apidocs/watson-data-api#listconnections



In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

## 1. Installation and Configuration of CPDCTL


### 1.1 Install the latest version of cpdctl

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

### 1.2 Check cpdctl version

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.79


### 1.3 Add CPD cluster configuration

In [5]:
CPD_USER_NAME = '<username>'
CPD_USER_PASSWORD = '<password>'
CPD_URL = '<cpd_url>'

Add "cpd35_user" user to the cpdctl configuration

In [6]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd35" cluster to the cpdctl configuration

In [7]:
! cpdctl config profile set cpd --url {CPD_URL} --user cpd_user

Add "cpd35" context to the cpdctl configuration

In [8]:
! cpdctl config context set cpd --profile cpd --user cpd_user

List available contexts

In [9]:
! cpdctl config context list

Name   Profile   User       Current   
cpd    cpd       cpd_user   *   


Make sure you use context which you prefer

In [10]:
! cpdctl config context use cpd

Switched to context "cpd".


List available projects in context

In [11]:
! cpdctl project list

...
ID                                     Name                                                 Created                    Description                                          Tags   
02351f31-ec56-4f83-b177-d5841d8bf162   Imported (project.9bc31a1b-11fb-49cb-8b31-3e52512…   2021-06-23T08:12:44.324Z   project for testing export                           []   
02ab599b-538e-46d0-b652-d435ba58ac8f   icp4dcatalogscenariocommon                           2021-06-16T18:12:58.265Z   The project for FVT test of the wkc catalog REST …   []   
04dbcac4-8877-40c1-ac1f-92055bed3eac   knish_mdi_test                                       2021-06-08T08:23:22.148Z                                                        []   
0a746aa5-3bfb-40a6-bdab-764dd4f4a932   MDI_Test_All_CONN                                    2021-06-17T09:30:16.401Z                                                        []   
13b302c3-d7d5-45c7-bcf1-d1c84bdaa9a4   icp4dcatalogscenariocommon                           2021-06-24T0

## 2. Demo cpdctl connections commands

### 2.0 Create a project for connection or Choose an existing project

To create a connection, first you need to have a project, catalog or space.
You can either choose an existing project:

In [12]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[0]" --raw-output
PROJECT_ID = result.s
print(f'project id: {PROJECT_ID}')

# You can also specify your project id directly:
# PROJECT_ID = "Your project ID"

project id: 02351f31-ec56-4f83-b177-d5841d8bf162


Or create a new project:

In [13]:
import uuid
STORAGE = {"type": "assetfiles", "guid": str(uuid.uuid4())}
STORAGE_JSON = json.dumps(STORAGE)
result = ! cpdctl project create --name "cpdctl-test-proj" --output json --raw-output --generator cli --storage '{STORAGE_JSON}' -j 'location'
PROJECT_ID = result.s.split('/')[-1]
print(f'project id: {PROJECT_ID}')

project id: c494d4b7-9e1c-45c3-b9d4-092dc6f1f024


### 2.1 Create a connection

Here, as an example, connection properties to dashdb were used. Type your connection properties.

In [14]:
CREATE_CONN_PROPERTIES = '''
{
      "database": "<database>",
      "host": "<host>",
      "password": "<password>",
      "username": "<username>"
}
'''

In [15]:
result = ! cpdctl connection create --name "cpdctl-test-conn" --description "test connection to dashdb" --datasource-type "dashdb" --project-id '{PROJECT_ID}' --properties '{CREATE_CONN_PROPERTIES}' -j metadata.asset_id --origin-country us --output json -j 'metadata.asset_id'
CONNECTION_ID = result.s[1:-1]
print(f'connection id: {CONNECTION_ID}')

connection id: 09771787-cc58-4404-b75d-6747c48b14b4


### 2.2 List connections

In [16]:
! cpdctl connection list --project-id '{PROJECT_ID}'

...
ID                                     Name               Description                 Type         Tags   
09771787-cc58-4404-b75d-6747c48b14b4   cpdctl-test-conn   test connection to dashdb   connection   []   


In [17]:
! cpdctl connection list --project-id '{PROJECT_ID}' --entity-name "cpdctl-test-conn"

...
ID                                     Name               Description                 Type         Tags   
09771787-cc58-4404-b75d-6747c48b14b4   cpdctl-test-conn   test connection to dashdb   connection   []   


In [18]:
! cpdctl connection list --project-id '{PROJECT_ID}' --entity-flags "restricted" --sort "entity.name"

...
ID                                     Name               Description                 Type         Tags   
09771787-cc58-4404-b75d-6747c48b14b4   cpdctl-test-conn   test connection to dashdb   connection   []   


### 2.3 Discover connection ad-hoc

In [19]:
! cpdctl connection discover-adhoc --path "/CONNUSER" --datasource-type "dashdb" --name "conn_dashdb_adhoc" --properties '{CREATE_CONN_PROPERTIES}'

...
                 
assets        <Array>   
first         <Nested Object>   
next          <Nested Object>   
path          /CONNUSER   
prev          <Nested Object>   
total_count   25   
asset_types   <Array>   


In [20]:
! cpdctl connection discover-adhoc --path "/CONNUSER" --datasource-type "dashdb" --name "conn_dashdb_adhoc" --properties '{CREATE_CONN_PROPERTIES}' --output json

{
  "asset_types": [
    {
      "dataset": true,
      "dataset_container": false,
      "type": "table"
    }
  ],
  "assets": [
    {
      "id": "FP_YS1DEV",
      "name": "FP_YS1DEV",
      "path": "/CONNUSER/FP_YS1DEV",
      "type": "table"
    },
    {
      "id": "FP_ypprod",
      "name": "FP_ypprod",
      "path": "/CONNUSER/FP_ypprod",
      "type": "table"
    },
    {
      "id": "FP_ypqa",
      "name": "FP_ypqa",
      "path": "/CONNUSER/FP_ypqa",
      "type": "table"
    },
    {
      "id": "FP_ys1dev",
      "name": "FP_ys1dev",
      "path": "/CONNUSER/FP_ys1dev",
      "type": "table"
    },
    {
      "id": "ISSUE43498",
      "name": "ISSUE43498",
      "path": "/CONNUSER/ISSUE43498",
      "type": "table"
    },
    {
      "id": "KMICHAL1",
      "name": "KMICHAL1",
      "path": "/CONNUSER/KMICHAL1",
      "type": "table"
    },
    {
      "id": "KMICHAL1SMALL",
      "name": "KMICHAL1SMALL",

      "type": "table"
    },
    {
      "id": "KMICHAL1_CUSTOME

### 2.4 Discover data asset (not available until CPD 3.5)

To discover data asset, we first need to have a connected data asset ID. 

To get it we may create a new connected data asset or take an existing data asset ID.


#### Creating new connected data asset

Replace properties with your data. Especially connection_path.

In [21]:
ASSET_METADATA = '''
{
      "name": "connected-data-asset",
      "asset_type": "data_asset",
      "origin_country": "us",
      "asset_category": "USER"
}
'''
ASSET_ENTITY = '''
{
        "data_asset": {      
            "mime_type": "text/csv",
            "dataset": false
        }
}
'''
ATTACHMENTS = '''
[
        {
            "asset_type": "data_asset",
            "connection_id": "replace_with_connection_id",
            "connection_path": "CONNUSER/product.csv"
        }
]
'''

ATTACHMENTS = ATTACHMENTS.replace("replace_with_connection_id", CONNECTION_ID)
result = ! cpdctl asset data-asset create --project-id '{PROJECT_ID}' --metadata '{ASSET_METADATA}' --entity '{ASSET_ENTITY}' --attachments '{ATTACHMENTS}' --output json -j metadata.asset_id
DATA_ASSET_ID = result.s[1:-1]
print(f'data asset id: {DATA_ASSET_ID}')

data asset id: 6ec48ebf-6cf3-464b-9705-16b809ab41ea


#### Discover data asset

In [22]:
! cpdctl connection discover-data-asset --data-asset-id '{DATA_ASSET_ID}' --project-id '{PROJECT_ID}'

...
first             path                   total_count   name                  type   
<Nested Object>   CONNUSER/product.csv   1             PRODUCT_NUMBER        <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             BASE_PRODUCT_NUMBER   <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             INTRODUCTION_DATE     <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             DISCONTINUED_DATE     <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             PRODUCT_TYPE_CODE     <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             PRODUCT_COLOR_CODE    <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             PRODUCT_SIZE_CODE     <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             PRODUCT_BRAND_CODE    <Nested Object>   
<Nested Object>   CONNUSER/product.csv   1             PRODUCT_IMAGE         <Nested Object>   


In [23]:
! cpdctl connection discover-data-asset --data-asset-id '{DATA_ASSET_ID}' --project-id '{PROJECT_ID}' --output json

{
  "fields": [
    {
      "name": "PRODUCT_NUMBER",
      "type": {
        "length": 10,
        "nullable": true,
        "scale": 0,
        "signed": true,
        "type": "integer"
      }
    },
    {
      "name": "BASE_PRODUCT_NUMBER",
      "type": {
        "length": 10,
        "nullable": true,
        "scale": 0,
        "signed": true,
        "type": "integer"
      }
    },
    {
      "name": "INTRODUCTION_DATE",
      "type": {
        "length": 26,
        "nullable": true,
        "scale": 6,
        "signed": false,
        "type": "timestamp"
      }
    },
    {
      "name": "DISCONTINUED_DATE",
      "type": {
        "length": 26,
        "nullable": true,
        "scale": 6,
        "signed": false,
        "type": "timestamp"
      }
    },
    {
      "name": "PRODUCT_TYPE_CODE",
      "type": {
        "length": 10,
        "nullable": true,
        "scale": 0,
        "signed": true,
        "type": "integer"
      }
    },
    {
      "name": "PRODUCT_

### 2.5 Get connection

In [24]:
! cpdctl connection get --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}'

...
                  
ID:            09771787-cc58-4404-b75d-6747c48b14b4   
Name:          cpdctl-test-conn   
Description:   test connection to dashdb   
Type:          connection   
Tags:          []   


### 2.6 Update connection

Here is an example of updating name of connection, fom existing to the new one

In [25]:
UPDATE_CONN_PROPERTIES = '''
[{
      "op": "replace",
      "path": "/name",
      "from": "cpdctl-test-conn",
      "value": "updated-conn-name"
}]
'''

! cpdctl connection update --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --patch-request '{UPDATE_CONN_PROPERTIES}'

...
                  
Name:          updated-conn-name   
Description:   test connection to dashdb   
Tags:          []   


### 2.7 List actions

In [26]:
! cpdctl connection list-actions --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}'

...
Name               Description   
get_record_count   Get the number of rows in the specified table   


In [27]:
! cpdctl connection list-actions --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --output json

{
  "actions": [
    {
      "description": "Get the number of rows in the specified table",
      "name": "get_record_count",
      "properties": {
        "input": [
          {
            "description": "Name of the schema that contains the table",
            "hidden": false,
            "label": "Schema name",
            "masked": false,
            "multiline": false,
            "name": "schema_name",
            "readonly": false,
            "required": false,
            "type": "string",
            "user_defined": false
          },
          {
            "description": "Name of the table for which to obtain the number of rows",
            "hidden": false,
            "label": "Table name",
            "masked": false,
            "multiline": false,
            "name": "table_name",
            "readonly": false,
            "required": true,
            "type": "string",
            "user_defined": false
          }
        ],
        "output": [

            "descrip

### 2.8 Perform action

Actions are different for each connector.
Action name with it's necassary properties can be taken from the list-actions output (look at 2.7 List actions)

In [28]:
PERFORM_ACTION_PROPS = '''
{
        "schema_name": "CONNUSER",
        "table_name": "product.csv"
}
'''

! cpdctl connection perform-action --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --action-name "get_record_count" --request='{PERFORM_ACTION_PROPS}'

...
                  
record_count   274   


### 2.9 Discover connection

In [29]:
! cpdctl connection discover --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --path="/CONNUSER"

...
                 
asset_types   <Array>   
assets        <Array>   
first         <Nested Object>   
path          /CONNUSER   
total_count   25   


In [30]:
! cpdctl connection discover --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --path="/CONNUSER" --output json

{
  "asset_types": [
    {
      "dataset": true,
      "dataset_container": false,
      "type": "table"
    }
  ],
  "assets": [
    {
      "id": "FP_YS1DEV",
      "name": "FP_YS1DEV",
      "path": "/CONNUSER/FP_YS1DEV",
      "type": "table"
    },
    {
      "id": "FP_ypprod",
      "name": "FP_ypprod",
      "path": "/CONNUSER/FP_ypprod",
      "type": "table"
    },
    {
      "id": "FP_ypqa",
      "name": "FP_ypqa",
      "path": "/CONNUSER/FP_ypqa",
      "type": "table"
    },
    {
      "id": "FP_ys1dev",
      "name": "FP_ys1dev",
      "path": "/CONNUSER/FP_ys1dev",
      "type": "table"
    },
    {
      "id": "ISSUE43498",
      "name": "ISSUE43498",
      "path": "/CONNUSER/ISSUE43498",
      "type": "table"
    },
    {
      "id": "KMICHAL1",
      "name": "KMICHAL1",
      "path": "/CONNUSER/KMICHAL1",
      "type": "table"
    },
    {
      "id": "KMICHAL1SMALL",
      "name": "KMICHAL1SMALL",

      "type": "table"
    },
    {
      "id": "KMICHAL1_CUSTOME

### 2.10 Delete connection

In [31]:
! cpdctl connection delete --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}'

...
OK


### 2.11 Clean up - Delete project created at the beggining (containing our connection)

In [32]:
! cpdctl project delete --project-id '{PROJECT_ID}'

...
OK


## 3. Demo cpdctl connection datasource types commands

### 3.1 List datasource types

In [33]:
! cpdctl connection datasource-type list

...
ID                                     Name                        Description                                          Type       Tags   
029e5d1c-ba73-4b09-b742-14c3a39b6cf9   informix                    IBM Informix database                                database   []   
048ed1bf-516c-46f0-ae90-fa3349d8bc1c   postgresql-ibmcloud         Databases for PostgreSQL database                    database   []   
05b7f0ea-6ae4-45e2-a455-cc280f110825   googlecloudstorage          Google Cloud Storage                                 file       []   
05c58384-862e-4597-b19a-c71ea7e760bc   impala                      Cloudera Impala database                             database   []   
06847b16-07b4-4415-a924-c63d11a17aa1   salesforce                  Salesforce.com                                       database   []   
0c431748-2572-11ea-978f-2e728ce88125   cosmos                      Microsoft Azure Cosmos DB                            file       []   
0cd4b64c-b485-47ed-a8c4-329c25412de

In [34]:
! cpdctl connection datasource-type list --sort entity.name --limit 5

...
ID                                     Name               Description                                          Type   Tags   
a0b1d14a-4767-404c-aac1-4ce0e62818c3   amazons3           Amazon Simple Storage Service (S3)                   file   []   
81bafdbd-b7c6-45c5-a4fd-6ec135f66f4e   assetfiles         Asset Files                                          file   []   
9a22e0af-8d19-4c4e-9aea-1d733e81315b   azureblobstorage   Microsoft Azure Blob Storage                         file   []   
6863060d-97c4-4653-abbe-958bde533f8c   azuredatalake      Microsoft Azure Data Lake Store via the WebHDFS A…   file   []   
2a7b4fa1-c770-4807-8871-a3c5def5aa2d   azurefilestorage   Microsoft Azure File Storage                         file   []   
Next token: 'https://internal-nginx-svc:12443/v2/datasource_types?offset=5&limit=5&sort=entity.name'


In [35]:
! cpdctl connection datasource-type list --entity-environment "private" --entity-product "igc"

...
ID                                     Name               Description                       Type       Tags   
0fd83fe5-8995-4e2e-a1be-679bb8813a6d   hive               Apache Hive database              database   []   
48695e79-6279-474a-b539-342625d3dfc2   sqlserver          Microsoft SQL Server database     database   []   
8c1a4480-1c29-4b33-9086-9cb799d7b157   db2                IBM Db2 database                  database   []   
8e65204d-6156-49e7-96e5-d635b2aa05f6   mongodb-ibmcloud   Databases for MongoDB database    database   []   
971223d3-093e-4957-8af9-a83181ee9dd9   oracle             Oracle database                   database   []   
c10e5224-f17d-4524-844f-e97b1305e489   hdfs-apache        Apache HDFS via the WebHDFS API   file       []   
c6fb9293-51eb-4f2b-b20c-4dafa3136744   mongodb            MongoDB database                  database   []   
cfdcb449-1204-44ba-baa6-9a8a878e6aa7   dashdb             Db2 Warehouse                     database   []   
e59b1c36-6f30

In [36]:
! cpdctl connection datasource-type list --connection-properties true --interaction-properties true --actions true

...
ID                                     Name                        Description                                          Type       Tags   
029e5d1c-ba73-4b09-b742-14c3a39b6cf9   informix                    IBM Informix database                                database   []   
048ed1bf-516c-46f0-ae90-fa3349d8bc1c   postgresql-ibmcloud         Databases for PostgreSQL database                    database   []   
05b7f0ea-6ae4-45e2-a455-cc280f110825   googlecloudstorage          Google Cloud Storage                                 file       []   
05c58384-862e-4597-b19a-c71ea7e760bc   impala                      Cloudera Impala database                             database   []   
06847b16-07b4-4415-a924-c63d11a17aa1   salesforce                  Salesforce.com                                       database   []   
0c431748-2572-11ea-978f-2e728ce88125   cosmos                      Microsoft Azure Cosmos DB                            file       []   
0cd4b64c-b485-47ed-a8c4-329c25412de

### 3.2 Get datasource type

In [37]:
! cpdctl connection datasource-type get --datasource-type dashdb

...
                  
ID:            cfdcb449-1204-44ba-baa6-9a8a878e6aa7   
Name:          dashdb   
Description:   Db2 Warehouse   
Type:          database   
Tags:          []   


In [38]:
! cpdctl connection datasource-type get --datasource-type dashdb --connection-properties true --interaction-properties true

...
                  
ID:            cfdcb449-1204-44ba-baa6-9a8a878e6aa7   
Name:          dashdb   
Description:   Db2 Warehouse   
Type:          database   
Tags:          []   
